# Paquetes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

# Parametros

In [3]:
# MODEL

_LANGUAGE_         = 'es'
_PRETRAINED_LM_    = 'pysentimiento/robertuito-base-cased'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.IA3Config() #transformers.ParallelConfig(mh_adapter = True, reduction_factor = 64)
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'parallel_adapter_checkPoints_es'
_LOGGING_STEPS_    = 2
_NUM_AUTHORS_      = [1, 3, 6, 12, 15, 30, 60, 90]


# TRAIN

_NO_GPUS_          = 1
_BATCH_SIZE_       = int(100 / _NO_GPUS_)
_EPOCHS_           = {'gender': 8, 'variety': 10}
_LEARNING_RATE_    = 1e-4

# Dataset

In [4]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'great britain' : 2,
                  'ireland'      : 3,
                  'new zealand'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [5]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [6]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
    
vocab = tokenizer.get_vocab()

In [7]:
# CREATE ONE INSTANCE PER DATA SPLIT

from DatasetPAN17 import BasePAN17, DatasetPAN17


baseTest  = BasePAN17(Dir              = '../data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_,
                      num_authors      = 200)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 56000



In [8]:
# CREATE DATA LOADER FOR EVERY TASK

tasks = ['gender']

# Model

In [9]:
from transformers import AutoAdapterModel


model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)

Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaAdapterModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at pysentimiento/robertuito-base-cased and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able

# Training and testing Adapters

In [10]:
from Training import train_model_with_adapters
from TestingPAN17 import test_model_with_adapters

num_v           = len(baseTest.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

accuracy = {}

for num in _NUM_AUTHORS_:
    # SHOW CURRENT PORTION
    print("Working with " + str(num) + " authors per label ... ")
    
    # GENERATES DATASET WITH CURRENT PORTION ----------------------
    baseTrain  = BasePAN17(Dir         = '../data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_,
                      num_authors      = num)
    dataset_dict = {}
    
    for task_name in tasks:
        
        dataset_dict[task_name] = DatasetPAN17(Base_Dataset = baseTrain, label = task_name)
        
        # ADD ADAPTER AND CLASSIFICATION HEAD----------------------
        model.add_adapter(
            adapter_name = task_name, 
            config       = _ADAPTER_CONFIG_
        )

        model.add_classification_head(
            head_name    = task_name,
            num_labels   = num_labels_dict[task_name],
          )

        
        # TRAIN ADAPTER--------------------------------------------
        train_model_with_adapters(model     = model,
                              dataset_dict  = dataset_dict,
                              epochs        = _EPOCHS_,
                              batch_size    = _BATCH_SIZE_,
                              no_gpus       = _NO_GPUS_,
                              output_dir    = _OUTPUT_DIR_,
                              logging_steps = _LOGGING_STEPS_,
                              learning_rate = _LEARNING_RATE_)
        
        # SAVE ACCURACY--------------------------------------------
        accuracy[num] = test_model_with_adapters(model, baseTest, task_name)
        print("accuracy with " + str(num) + " authors per label: " + str(accuracy[num]))
        
        # SAVE ADAPTER AND DELETE----------------------------------
        model.save_adapter("num_authors" + str(num), task_name)
        model.delete_adapter(task_name)
        model.delete_head(task_name)

Working with 1 authors per label ... 

Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 280



/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 280
  Num Epochs = 8
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 24


Step,Training Loss
2,0.704800
4,0.628000
6,0.579800
8,0.515600
10,0.509100
12,0.493900
14,0.451600
16,0.426100
18,0.424800
20,0.400400




Training completed. Do not forget to share your model on huggingface.co/models =)


acc: 0.635: 100%|███████████████████████████████████████████████████████████████████| 2800/2800 [11:55<00:00,  3.92it/s]
Configuration saved in num_authors1/adapter_config.json
Module weights saved in num_authors1/pytorch_adapter.bin
Configuration saved in num_authors1/head_config.json
Module weights saved in num_authors1/pytorch_model_head.bin


accuracy with 1 authors per label: 0.635
Working with 3 authors per label ... 

Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...


...

accuracy with 60 authors per label: 0.7678571428571429
Working with 90 authors per label ... 

Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 25200



Adding adapter 'gender'.
Adding head 'gender' with config {'head_type': 'classification', 'num_labels': 2, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'LABEL_0': 0, 'LABEL_1': 1}, 'use_pooler': False, 'bias': True}.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25200
  Num Epochs = 8
  Instantaneous batch size per device = 100
  Total train b

Step,Training Loss
2,0.715200
4,0.695700
6,0.669900
8,0.673400
10,0.670600
12,0.671900
14,0.641400
16,0.639000
18,0.669500
20,0.662800


Saving model checkpoint to parallel_adapter_checkPoints_es/gender/checkpoint-500
Configuration saved in parallel_adapter_checkPoints_es/gender/checkpoint-500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_es/gender/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_es/gender/checkpoint-500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_es/gender/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_es/gender/checkpoint-500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_es/gender/checkpoint-500/gender/pytorch_model_head.bin
Saving model checkpoint to parallel_adapter_checkPoints_es/gender/checkpoint-1000
Configuration saved in parallel_adapter_checkPoints_es/gender/checkpoint-1000/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_es/gender/checkpoint-1000/gender/pytorch_adapter.bin

accuracy with 90 authors per label: 0.7767857142857143


In [11]:
accuracy

{1: 0.635,
 3: 0.67,
 6: 0.7164285714285714,
 12: 0.7328571428571429,
 15: 0.735,
 30: 0.7460714285714286,
 60: 0.7678571428571429,
 90: 0.7767857142857143}